#### REGULARIZATION

We want to create two prediction models both for solar energy output and wind energy output. In order to predict these outputs we want to build a model that selects the best possible predictors maximizing the feature significance, avoiding colinearity, ensuring sparisty, considering the best feature transformations... To do so we will apply different models like Lasso, and Ridge and then we can apply Holistic Regression to see how the model is improved by it. We will run all of these models by all predictors of solar and energy output. 

In [111]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, DataFrames
#read train and validation data and split it into X and y
X_train_solar=Matrix(DataFrame(CSV.File("X_train_solar.csv")))
y_train_solar=Matrix(DataFrame(CSV.File("y_train_solar.csv")))
X_train_wind=Matrix(DataFrame(CSV.File("X_train_wind.csv")))
y_train_wind=Matrix(DataFrame(CSV.File("y_train_wind.csv")))

X_valid_solar=Matrix(DataFrame(CSV.File("X_valid_solar.csv")))
y_valid_solar=Matrix(DataFrame(CSV.File("y_valid_solar.csv")))
X_valid_wind=Matrix(DataFrame(CSV.File("X_valid_wind.csv")))
y_valid_wind=Matrix(DataFrame(CSV.File("y_valid_wind.csv")));

### Functions

In [112]:
function lasso(X,y,lambda=0.25)
    
    #Build model
    model = Model(Gurobi.Optimizer)#we have defined the model, pass Gurobi optimizer into the model
    set_optimizer_attribute(model,"OutputFlag",0)
    
    #Insert variables
    n, p = size(X)
    
    @variable(model, beta[j=1:p])
    @variable(model, beta_abs[j=1:p])
    
    
    #Insert constraints

    @constraint(model, beta_abs .>= beta) #put the dot is like doing the loop over all j
    @constraint(model, beta_abs .>= -beta)
    
    
    #sum((y-X*beta).^2)) = sum((y[i] - sum(X[i,j]*beta[j]) for j=1:p)^2 for i=1:n)
    
    #Insert objective
    @objective(model, Min, sum((y-X*beta).^2) + lambda*sum(beta_abs))
    
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
    
end

lasso (generic function with 2 methods)

In [113]:
function mse(X,y, beta)
    return sum((y-X*beta).^2)/length(y)
end

mse (generic function with 1 method)

In [114]:
function correlation(X,p)
    correlation_matrix = Statistics.cor(X) #compute corr matrix
    n,m = size(correlation_matrix) #set sizes
    correlated_pairs=[] #empty list to store correlated pairs
    for i=1:n #for all features
        for j=i+1:m 
            if abs(correlation_matrix[i,j])>p #if the abs value of the corr is higher than p
                push!(correlated_pairs, (i,j)) #append pair to list of correlated pairs
            end
        end
    end
    return correlated_pairs
end

correlation (generic function with 1 method)

In [115]:
#Build transformation function
function transformation(X)
    X_old=DataFrame(X, :auto) #define X as a df
    X_new=DataFrame() #new empty df 
    n,p=size(X_old)
    e=1
    for i=1:p #for each feature in X add 4 transformations
        X_new[!, "X$i"]=X_old[:,i] #transformation 1: no transformation
        X_new[!, "Sqrt$i"]=X_old[:,i].^2 #transformation 2: square root
        X_new[!, "Log$i"]=log.(abs.(X_old[:,i])) #transformation 3: log
        X_new[!, "Exp$i"]=exp.(X_old[:,i]) #transformation 4: exp
        X_new[!, "Abs_Sqrt$i"]=sqrt.(abs.(X_old[:,i])) #transformation 3: absolute squared root
    end
    return(X_new) #we return a new df with all transformations, it will have size nxp*4
end

transformation (generic function with 1 method)

In [116]:
#Build transformation function
function transformation(X)
    X_old=DataFrame(X, :auto) #define X as a df
    X_new=DataFrame() #new empty df 
    n,p=size(X_old)
    e=1
    for i=1:p #for each feature in X add 4 transformations
        X_new[!, "X$i"]=X_old[:,i] #transformation 1
        X_new[!, "Sqrt$i"]=X_old[:,i].^2 #transformation 2
        X_new[!, "Abs_Sqrt$i"]=sqrt.(abs.(X_old[:,i])) #transformation 3
        X_new[!, "Log$i"]=log.(abs.(X_old[:,i]).+e) #transformation 4
    end
    return(X_new) #we return a new df with all transformations, it will have size nxp*4
end

transformation (generic function with 1 method)

In [163]:
function holistic_regression(X,y,lambda=0.25, per=0.6, M=50, k=10) #add parameter per
    
    #Call functions
    X_new=Matrix(transformation(X)) #call function with all transformations of X
    HC = correlation(X_new, per) #call correlation function to compute hc_pairs
    
    #Set sizes
    n,p_new=size(X_new)
    
    #Build model
    model = Model(Gurobi.Optimizer)
    #model = Model(Gurobi.Optimizer, NonConvex = 2)#we have defined the model, pass Gurobi optimizer into the model
    #model = Model(with_optimizer(Gurobi.Optimizer, NonConvex = 2))
    set_optimizer_attribute(model,"OutputFlag",0)
    
    #Insert variables
    @variable(model, beta[1:p_new])
    @variable(model, beta_abs[1:p_new])
    @variable(model, z[1:p_new], Bin) #we add a binary variable
    

    #Insert constraints
    @constraint(model, beta_abs .>= beta) #put the dot is like doing the loop over all j
    @constraint(model, beta_abs .>= -beta)
    
        
    #sparsity constraint: over all 60 features (including transformations) 
    @constraint(model, -M*z .<= beta)
    @constraint(model, beta .<= M*z)
    @constraint(model, sum(z) <= k) 
    
    #constraint on Transformation: from the 4 transformations per each feature we only select one
    for i=1:4:p_new
        #for j=i:i+3 #for every 4 transformations
        @constraint(model, sum(z[i:i+3])<=1)
        #end
    end #we get a vector with 15 features
    
    #constraint on HC pairs once we have selected 15 features 
    for (i,j) in HC
        @constraint(model, z[i] + z[j] <= 1)
    end #we can only take one of the pairs of correlated pairs
    #@constraint(model, sum(z[i])<=k) #ensure that the model has at most 8 features
    
        
    #Insert objective
    @objective(model, Min, sum((y-X_new*beta).^2) + lambda*sum(beta_abs))
    
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
    
end

holistic_regression (generic function with 5 methods)

### NORMALIZE DATA
We normalize the data with min max scaling. We apply normalization for the numerical features only and then we use alpha to balance between numerical and binary variables.

In [120]:
function min_max_scaling(X, num_feature_indices)
    for i in num_feature_indices
        X[:, i] = (X[:, i] .- minimum(X[:, i])) ./ (maximum(X[:, i]) .- minimum(X[:, i]))
    end
    return X
end

min_max_scaling (generic function with 1 method)

#### SOLAR

In [139]:
#scale SOLAR TRAIN only numerical
X_train_solar=min_max_scaling(Matrix(X_train_solar), 1:15)

#numerical features
X_num_solar = X_train_solar[:, 1:15]

# Binary features as they are
X_bin_solar = X_train_solar[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_solar = alpha*X_num_solar
X_bin_solar = (1-alpha)*X_bin_solar;

# Append data
X_train_solar_norm = [X_num_solar X_bin_solar];

In [138]:
#scale SOLAR TRAIN only numerical
X_valid_solar=min_max_scaling(Matrix(X_valid_solar), 1:15)

#numerical features
X_num_solar = X_valid_solar[:, 1:15]

# Binary features as they are
X_bin_solar = X_valid_solar[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_solar = alpha*X_num_solar
X_bin_solar = (1-alpha)*X_bin_solar;

# Append data
X_valid_solar_norm = [X_num_solar X_bin_solar];

#### WIND

In [140]:
X_train_wind=min_max_scaling(Matrix(X_train_wind), 1:15)

#numerical features
X_num_wind = X_train_wind[:, 1:15]

# Binary features as they are
X_bin_wind = X_train_wind[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_wind = alpha*X_num_wind
X_bin_wind = (1-alpha)*X_bin_wind;

# Append data
X_train_wind_norm = [X_num_wind X_bin_wind];

In [141]:
X_valid_wind=min_max_scaling(Matrix(X_valid_wind), 1:15)

#numerical features
X_num_wind = X_valid_wind[:, 1:15]

# Binary features as they are
X_bin_wind = X_valid_wind[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_wind = alpha*X_num_wind
X_bin_wind = (1-alpha)*X_bin_wind;

# Append data
X_valid_wind_norm = [X_num_wind X_bin_wind];

### SOLAR ENERGY OUTPUT

In [143]:
beta_lasso=lasso(X_train_solar_norm,y_train_solar_norm,0.25)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


26-element Vector{Float64}:
  0.002364977583234187
  1.0230128719423987e-10
  8.154002852913467e-13
  0.0037859301103054166
  4.738885373264396e-13
  0.0037823182066607035
  2.3893180524601933e-13
  0.0010317647995062666
  1.0017770179074741e-7
  0.0011959414515760034
  0.00472827930846247
  0.0013377830313550283
  0.00388054694169201
  0.0026789810295118933
  8.328183709055352e-9
  1.7635237924202087e-12
  1.0531825202719455e-12
  1.0694606986792117e-11
 -6.354453883195101e-13
 -1.8552301931506344e-13
  1.0308680733877795e-12
  1.859352798170065e-12
  0.010548539138471742
 -8.345836167189308e-13
  1.859352798170065e-12
  1.0301748159414565e-12

In [144]:
mse(X_valid_solar_norm, y_valid_solar_norm, beta_lasso)

0.00016719467906875534

In [148]:
beta_opt=holistic_regression(X_train_solar_norm, y_train_solar_norm, 0.2, 0.6, 50, 10)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


104-element Vector{Float64}:
 0.0
 0.0
 0.0022847297510582873
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [150]:
#return indices of selected features
selected_features=findall(x->x>0, beta_opt)

10-element Vector{Int64}:
  3
 15
 23
 29
 35
 39
 43
 47
 51
 55

In [151]:
#what is the name of the selected features
names(transformation(X_train_solar))[selected_features]

10-element Vector{String}:
 "Abs_Sqrt1"
 "Abs_Sqrt4"
 "Abs_Sqrt6"
 "X8"
 "Abs_Sqrt9"
 "Abs_Sqrt10"
 "Abs_Sqrt11"
 "Abs_Sqrt12"
 "Abs_Sqrt13"
 "Abs_Sqrt14"

* Absolute squared value of temperature
* Absolute squared value of sealevelpressure
* Absolute squared value of solarirradiation

In [153]:
#report mse
X_valid_solar_norm_trans=Matrix(transformation(X_valid_solar_norm))
mse(X_valid_solar_norm_trans, y_valid_solar_norm, beta_opt)

0.0001539088840623817

                DO CROSS VALIDATION TO SEE WHICH HYPERPARAMETERS ARE BETTER FOR HOLISTIC REGRESSION!

### WIND ENERGY OUTPUT

In [154]:
beta_lasso=lasso(X_train_wind_norm,y_train_wind_norm,0.25)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


26-element Vector{Float64}:
 -5.9195148853837106e-12
 -9.707937568958661e-12
 -0.004641564285849145
 -1.3022976711578445e-9
  3.0667225224952805e-12
  0.002001954324278266
 -6.941982102736347e-13
 -0.0018259408045796569
  0.08658460529742229
  0.0048768284982343875
 -0.00502490324086349
 -0.0017505624678616778
  0.00891529039369091
 -0.00936388328852403
 -1.061720625543581e-8
  7.014151095538227e-11
 -0.00019893076778380558
  1.3647793639538894e-8
  1.1390441454357012e-11
 -6.785032768247417e-14
 -1.705909998788249e-12
 -6.380218356754201e-13
  1.480254502946299e-11
  1.0733115689064553e-11
 -6.380218356754201e-13
 -1.705909998788249e-12

In [155]:
mse(X_valid_wind_norm, y_valid_wind_norm, beta_lasso)

0.00028558937652363415

### CROSS VALIDATION: best lambda, rho, and k

In [173]:
#find best lambda
lowest_mse=Inf
best_lambda=0
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, i, 0.6, 50, 10)
    X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
    mse_temp= mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)
    if mse_temp<lowest_mse
        lowest_mse=mse_temp
        best_lambda=i
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


In [175]:
#find best lambda and best rho
lowest_mse=Inf
best_lambda=0
best_rho=0
for lambda in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    for rho in [0.5, 0.6, 0.7, 0.8, 0.9]
        beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, lambda, rho, 50, 10)
        X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
        mse_temp= mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)
        #for each lambda, find the best rho
        if mse_temp<lowest_mse
            lowest_mse=mse_temp
            best_lambda=lambda
            best_rho=rho
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19

In [179]:
#find best lambda, rho, and k
lowest_mse=Inf
best_lambda=0
best_rho=0
best_k=0
for lambda in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    for rho in [0.5, 0.6, 0.7, 0.8, 0.9]
        for k in [6,7,8,9,10,11,12,13]
            beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, lambda, rho, 50, k)
            X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
            mse_temp= mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)
            #for each lambda, find the best rho
            if mse_temp<lowest_mse
                lowest_mse=mse_temp
                best_lambda=lambda
                best_rho=rho
                best_k=k
            end
        end
    end
end

In [ ]:
println("Best lambda: ", best_lambda)
println("Best rho: ", best_rho)
println("Best k: ", best_k)

In [164]:
#holistic regression
beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, 0.1, 0.45, 50, 10)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


104-element Vector{Float64}:
  0.0
 -0.00781142665320559
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [158]:
#return indices of selected features
selected_features=findall(x->x>0, beta_opt)

4-element Vector{Int64}:
 23
 36
 39
 51

In [159]:
#what is the name of the features?
names(transformation(X_train_wind))[selected_features]

4-element Vector{String}:
 "Abs_Sqrt6"
 "Log9"
 "Abs_Sqrt10"
 "Abs_Sqrt13"

* Absolute value of windspeed
* Absolute value of winddirection
* Absolute value of precipitation type rain


In [160]:
#mse
X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)

0.0002889647778727034